# Install openai-agents SDK

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.1/571.1 kB 23.8 MB/s eta 0:00:00


# Make your Jupyter Notebook capable of running asynchronous functions.

In [2]:
import nest_asyncio
nest_asyncio.apply()

# Run Google Gemini with OPENAI-Agent SDK

In [3]:
import os

from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata

In [4]:
from agents import (
    Agent,
    Runner,
    function_tool,
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled,
)

BASE_URL = os.getenv("EXAMPLE_BASE_URL") or "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = os.getenv("EXAMPLE_API_KEY") or userdata.get("GEMINI_API_KEY")
MODEL_NAME = os.getenv("EXAMPLE_MODEL_NAME") or "gemini-2.0-flash"


if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError(
        "Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code."
    )



client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

set_default_openai_client(client=client, use_for_tracing=True)
set_default_openai_api("chat_completions")
# set_tracing_disabled(disabled=True)

> ** Note ** Set up the custom trace processor
```ptyhon
local_processor = LocalTraceProcessor()
set_trace_processors([local_processor])
```

In [5]:
import os
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, set_default_openai_api, set_default_openai_client, set_trace_processors
from agents.tracing.processor_interface import TracingProcessor
from pprint import pprint
# Custom trace processor to collect trace data locally
class LocalTraceProcessor(TracingProcessor):
    def __init__(self):
        self.traces = []
        self.spans = []

    def on_trace_start(self, trace):
        self.traces.append(trace)
        print(f"Trace started: {trace.trace_id}")

    def on_trace_end(self, trace):
        print(f"Trace ended: {trace.export()}")

    def on_span_start(self, span):
        self.spans.append(span)
        print(f"Span started: {span.span_id}")
        print(f"Span details: ")
        pprint(span.export())

    def on_span_end(self, span):
        print(f"Span ended: {span.span_id}")
        print(f"Span details:")
        pprint(span.export())

    def force_flush(self):
        print("Forcing flush of trace data")

    def shutdown(self):
        print("=======Shutting down trace processor========")
        # Print all collected trace and span data
        print("Collected Traces:")
        for trace in self.traces:
            print(trace.export())
        print("Collected Spans:")
        for span in self.spans:
            print(span.export())


if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

# Create OpenAI client
client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

# Configure the client
set_default_openai_client(client=client, use_for_tracing=True)
set_default_openai_api("chat_completions")

# Set up the custom trace processor
local_processor = LocalTraceProcessor()
set_trace_processors([local_processor])

# Example function to run an agent and collect traces
async def main():
    agent = Agent(name="Example Agent", instructions="Perform example tasks.", model=MODEL_NAME)

    with trace("Example workflow"):
        first_result = await Runner.run(agent, "Start the task")
        second_result = await Runner.run(agent, f"Rate this result: {first_result.final_output}")
        print(f"Result: {first_result.final_output}")
        print(f"Rating: {second_result.final_output}")

# Run the main function
import asyncio
asyncio.run(main())

Trace started: trace_7c0bd262abab4924a3b8fce47d7dbb3a
Span started: span_7c84732e973d47b38161f62c
Span details: 
{'ended_at': None,
 'error': None,
 'id': 'span_7c84732e973d47b38161f62c',
 'object': 'trace.span',
 'parent_id': None,
 'span_data': {'handoffs': [],
               'name': 'Example Agent',
               'output_type': 'str',
               'tools': [],
               'type': 'agent'},
 'started_at': '2025-03-19T05:30:35.033805+00:00',
 'trace_id': 'trace_7c0bd262abab4924a3b8fce47d7dbb3a'}
Span started: span_0ecd871eb0a945a8aed68b1c
Span details: 
{'ended_at': None,
 'error': None,
 'id': 'span_0ecd871eb0a945a8aed68b1c',
 'object': 'trace.span',
 'parent_id': 'span_7c84732e973d47b38161f62c',
 'span_data': {'input': None,
               'model': 'gemini-2.0-flash',
               'model_config': {'base_url': 'https://generativelanguage.googleapis.com/v1beta/openai/',
                                'frequency_penalty': None,
                                'max_tokens': Non